# CONFLUENCE Tutorial: Continental-Scale Modeling - South America

This notebook demonstrates how to set up a continental-scale SUMMA model for South America. We'll move quickly through the workflow, focusing on the scale differences from previous tutorials.

## Key Points
- **Scale**: From country (Iceland) to continent (South America)
- **Computational considerations**: Much larger domain

## 1. Quick Setup

In [1]:
# Import required libraries
import sys
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import box
import contextily as cx
from datetime import datetime
import xarray as xr

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE
from CONFLUENCE import CONFLUENCE

plt.style.use('default')
%matplotlib inline

print(f"Working from: {confluence_path}")

Working from: /home/darri.eythorsson/code/CONFLUENCE


## 2. Initialize CONFLUENCE for Continental Domain
We'll configure CONFLUENCE for a continental-scale domain with appropriate settings for North America.

In [2]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Load template configuration
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_South_America.yaml'
with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update with South America-specific settings
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Set South America domain and continental-specific settings
config_dict['DOMAIN_NAME'] = "South_America_tutorial"  
config_dict['EXPERIMENT_ID'] = "continental_run_1"
config_dict['EXPERIMENT_TIME_START'] = "2018-01-01 01:00"
config_dict['EXPERIMENT_TIME_END'] = "2018-02-31 23:00"  
config_dict['SPATIAL_MODE'] = "Distributed"

# North America continent regional domain settings
config_dict['BOUNDING_BOX_COORDS'] = "83.0/-170.0/5.0/-50.0"  # North/West/South/East
config_dict['DELINEATE_BY_POURPOINT'] = False
config_dict['DELINEATE_COASTAL_WATERSHEDS'] = True
config_dict['DOMAIN_DEFINITION_METHOD'] = "delineate" 
config_dict['STREAM_THRESHOLD'] = 7500  # Larger threshold for continental scale
config_dict['DOMAIN_DISCRETIZATION'] = "GRUs"
config_dict['MPI_PROCESSES'] = 1  # Higher for parallel processing - make sure this matches the number of cores requested in your allocation 

# Write to temporary config file
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_SA_template.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f)

# Initialize CONFLUENCE
confluence = CONFLUENCE(temp_config_path)


# Display configuration
print("=== North America Continental Configuration ===")
print(f"Domain Name: {confluence.config['DOMAIN_NAME']}")
print(f"Bounding Box: {confluence.config['BOUNDING_BOX_COORDS']}")
print(f"Delineate by Pour Point: {confluence.config['DELINEATE_BY_POURPOINT']} (Full continent!)")
print(f"Include Coastal Watersheds: {confluence.config['DELINEATE_COASTAL_WATERSHEDS']}")
print(f"Stream Threshold: {confluence.config['STREAM_THRESHOLD']} (larger for continental scale)")
print(f"Min GRU Size: {confluence.config['MIN_GRU_SIZE']} km²")
print(f"MPI Processes: {confluence.config['MPI_PROCESSES']} (high for parallel processing)")

15:36:22 - INFO - ============================================================


2025-05-13 15:36:22,596 - confluence_general - INFO - ============================================================


15:36:22 - INFO - CONFLUENCE Logging Initialized


2025-05-13 15:36:22,599 - confluence_general - INFO - CONFLUENCE Logging Initialized


15:36:22 - INFO - Domain: South_America_tutorial


...

15:36:22 - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


2025-05-13 15:36:22,698 - confluence_general - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


15:36:23 - INFO - CONFLUENCE system initialized successfully


2025-05-13 15:36:23,077 - confluence_general - INFO - CONFLUENCE system initialized successfully


=== North America Continental Configuration ===
Domain Name: South_America_tutorial
Bounding Box: 83.0/-170.0/5.0/-50.0
Delineate by Pour Point: False (Full continent!)
Include Coastal Watersheds: True
Stream Threshold: 7500 (larger for continental scale)
Min GRU Size: 0 km²
MPI Processes: 1 (high for parallel processing)


## 3. Project Setup


In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")
print("Setting up continental project...")

# Setup project
project_dir = confluence.managers['project'].setup_project()
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

print("\nDirectory purposes:")
print("  📁 shapefiles: Domain geometry (thousands of watersheds, river networks)")
print("  📁 attributes: Static characteristics (elevation, soil, land cover)")
print("  📁 forcing: Meteorological inputs (precipitation, temperature)")
print("  📁 simulations: Model outputs")
print("  📁 evaluation: Performance metrics and comparisons")
print("  📁 plots: Visualizations")


## 4. Geospatial Domain Definition - Continental Data Acquisition
We need to acquire continental-scale geospatial data, which will be much larger than for regional domains.

In [ ]:
# Step 2: Geospatial Domain Definition and Analysis
print("=== Step 2: Geospatial Domain Definition and Analysis ===")

# Acquire attributes
print("Acquiring continental-scale attributes (DEM, soil, land cover)...")
print("Note: This downloads LARGE datasets")
confluence.managers['data'].acquire_attributes()
print("✓ Continental attributes acquired\n")

## 5. Continental Domain Delineation
Now we'll delineate the entire continent, creating thousands of watersheds. This is computationally intensive.

In [ ]:
# Define continental domain
print("Delineating continental domain...")
print(f"Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
print(f"Stream threshold: {confluence.config['STREAM_THRESHOLD']} (high for continent)")
print(f"MPI processes: {confluence.config['MPI_PROCESSES']} (parallel processing)")
print("This creates thousands of watersheds across North America...")
print("⚠️ Warning: This step may take several hours on high-performance computing resources")

watershed_path = confluence.managers['domain'].define_domain()

# Check results
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

basin_count = 0
if basin_path.exists():
    basin_files = list(basin_path.glob('*.shp'))
    if basin_files:
        basins = gpd.read_file(basin_files[0])
        basin_count = len(basins)
        print(f"\n✓ Created {basin_count} watersheds across North America")
        print(f"Total area: {basins.geometry.area.sum() / 1e6:.0f} km²")

network_count = 0
if network_path.exists():
    network_files = list(network_path.glob('*.shp'))
    if network_files:
        rivers = gpd.read_file(network_files[0])
        network_count = len(rivers)
        print(f"✓ Created river network with {network_count} segments")

Delineating continental domain...
Method: delineate
Stream threshold: 7500 (high for continent)
MPI processes: 1 (parallel processing)
This creates thousands of watersheds across North America...
⚠️ Warning: This step may take several hours on high-performance computing resources
15:36:23 - INFO - Domain definition workflow starting with: delineate


2025-05-13 15:36:23,100 - confluence_general - INFO - Domain definition workflow starting with: delineate


15:36:23 - INFO - Starting geofabric delineation


2025-05-13 15:36:23,110 - confluence_general - INFO - Starting geofabric delineation


15:36:23 - INFO - Geofabric delineation completed successfully


...

2025-05-13 15:38:04,824 - confluence_general - ERROR - Error executing command: mpirun -n 1 /work/comphyd_lab/data/CONFLUENCE_data/installs/TauDEM/bin/streamnet -fel /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-fel.tif -p /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-fdir.tif -ad8 /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-ad8.tif -src /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-src.tif -ord /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-ord.tif -tree /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/basin-tree.dat -coord /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/basin-coord.dat -net /work/comphyd_lab/data/CONFLUENCE_data/domain_South_Ame

15:38:04 - ERROR - Error details: ERROR 4: /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-src.tif: No such file or directory
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 21.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------



2025-05-13 15:38:04,827 - confluence_general - ERROR - Error details: ERROR 4: /work/comphyd_lab/data/CONFLUENCE_data/domain_South_America_tutorial/taudem-interim-files/d8/elv-src.tif: No such file or directory
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 21.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------



15:38:04 - INFO - Retrying in 5 seconds...


2025-05-13 15:38:04,830 - confluence_general - INFO - Retrying in 5 seconds...


## 7. Continental Watershed Discretization
Now we need to discretize our continental domain into GRUs and HRUs, which will create tens of thousands of computational units.

In [ ]:
# Discretize continental domain
print(f"Creating continental HRUs using method: {confluence.config['DOMAIN_DISCRETIZATION']}")
print("⚠️ Warning: This step may take many hours and require significant memory")
print(f"Minimum GRU size: {confluence.config['MIN_GRU_SIZE']} km² (larger than regional domain)")

hru_path = confluence.managers['domain'].discretize_domain()

# Check results
hru_path = project_dir / 'shapefiles' / 'catchment'
if hru_path.exists():
    hru_files = list(hru_path.glob('*.shp'))
    if hru_files:
        # Note: For continental scale, we might not want to load all HRUs at once
        print("\n⚠️ Continental HRU file may be very large. Loading sample statistics instead.")
        
        # Get basic file stats without loading entire shapefile
        hru_file_size = hru_files[0].stat().st_size / (1024**2)  # Size in MB
        print(f"HRU shapefile size: {hru_file_size:.1f} MB")
        
        # Option to load a small sample of HRUs for statistics
        print("Loading small sample of HRUs for statistics...")
        sample_size = min(1000, basin_count)  # Limit sample size
        hru_sample = gpd.read_file(hru_files[0], rows=slice(0, sample_size))
        
        print(f"Sample contains {len(hru_sample)} HRUs")
        sample_grus = hru_sample['GRU_ID'].nunique()
        print(f"GRUs in sample: {sample_grus}")
        
        # Estimate total counts
        if basin_count > 0 and sample_grus > 0:
            est_hru_total = len(hru_sample) * (basin_count / sample_grus)
            print(f"Estimated total HRUs: ~{est_hru_total:.0f} (based on sample)")
        
        print("\nContinental statistics:")
        print(f"  - Total watersheds (GRUs): ~{basin_count}")
        print(f"  - Computational units (HRUs): Tens to hundreds of thousands")
        print(f"  - Domain extent: {abs(lat_max - lat_min):.1f}° latitude × {abs(lon_max - lon_min):.1f}° longitude")

## 8. Model Agnostic Processing - Continental Forcing Data
For continental domains, forcing data acquisition and processing is particularly data-intensive.

In [ ]:
# Step 3: Model Agnostic Data Pre-Processing
print("=== Step 3: Model Agnostic Data Pre-Processing ===")

# Process observed data
print("Processing observed streamflow data...")
print("Note: For continental modeling, we often use a subset of gauged watersheds for evaluation")
confluence.managers['data'].process_observed_data()

# Acquire continental-scale forcings
print(f"\nAcquiring continental forcing data: {confluence.config['FORCING_DATASET']}")
print("Expected data size: 10s to 100s of GB")
print(f"Period: {confluence.config['EXPERIMENT_TIME_START']} to {confluence.config['EXPERIMENT_TIME_END']}")
print("⚠️ Warning: This step will take several hours and require significant storage")
confluence.managers['data'].acquire_forcings()

# Run model-agnostic preprocessing
print("\nRunning continental-scale model-agnostic preprocessing...")
print("This step remaps climate data to tens of thousands of HRUs")
print("⚠️ Warning: High memory requirements (10s of GB)")
confluence.managers['data'].run_model_agnostic_preprocessing()

## 9. Model-Specific Preprocessing for Continental Domain
Preparing model input files for continental-scale modeling presents unique challenges.

In [ ]:
# Step 4: Model Specific Processing and Initialization
print("=== Step 4: Model Specific Processing and Initialization ===")

# Preprocess models
print(f"Preparing continental-scale {confluence.config['HYDROLOGICAL_MODEL']} input files...")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Routing: {confluence.config['ROUTING_MODEL']}")
print("⚠️ Warning: This will create very large input files")
print("Expected file sizes: Several GB per input file")
confluence.managers['model'].preprocess_models()

print("\n=== Continental Model Configuration Complete ===")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")
print(f"Number of GRUs: ~{basin_count}")
print("Number of HRUs: Tens to hundreds of thousands")
print("\nModel is now ready for execution with high-performance computing resources")

## 10. Continental Model Running Considerations
For demonstration, we'll discuss running a continental model without actually running it, as it would require significant computational resources.

In [ ]:
# Note: We don't actually run the continental model here
print("=== Continental Model Execution Considerations ===")
print("Running a continental-scale model requires significant HPC resources.")
print("\nTo run the model (when adequate resources are available):")
print("  confluence.managers['model'].run_models()")

print("\nTypical resource requirements:")
print("  - Memory: 100+ GB RAM")
print("  - CPU: 40+ cores for parallel processing")
print("  - Storage: 1+ TB for inputs/outputs")
print("  - Runtime: Days to weeks")

print("\nCommon execution strategies:")
print("  1. Break continent into regions and run separately")
print("  2. Use MPI for massive parallelization")
print("  3. Run shorter test periods before full simulation")
print("  4. Use HPC job scheduling for long-running simulations")

print("\nFor this tutorial, we've completed the continental model setup")
print("without running the full model due to computational constraints.")

## 11. Continental-Scale Considerations

### Computational Requirements
- **Memory**: 100+ GB for data processing
- **Storage**: 1+ TB for inputs/outputs
- **CPU**: High parallelization (40+ cores)
- **Runtime**: Days to weeks for full simulations

### Key Configuration Differences
```yaml
STREAM_THRESHOLD: 7500        # Higher for continental scale
MPI_PROCESSES: 40            # More parallel processes
MIN_GRU_SIZE: 50             # Larger minimum size to manage computational load
FORCING_DATASET: ERA5        # Global reanalysis data
```

### Challenges at Continental Scale
1. **Data Volume**: TB of input/output data
2. **Heterogeneity**: Diverse climates, terrains, vegetation
3. **Calibration**: How to calibrate thousands of watersheds?
4. **Validation**: Limited observations for many basins
5. **Computation**: Requires HPC resources

### Use Cases
- Climate change impact assessment
- Continental water balance
- Large-scale flood forecasting
- Water resources planning
- Earth system modeling

## 12. Summary
Let's summarize what we've accomplished with our continental-scale setup.

In [ ]:
# Final summary
print("=== Continental Model Setup Complete ===\n")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")
print(f"Scale: Continental (~24.7 million km²)")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Status: Ready for simulation")

print("\nScale progression in tutorials:")
print("  1. Watershed: Bow at Banff (~2,200 km²)")
print("  2. Country: Iceland (~103,000 km²)")
print("  3. Continent: North America (~24,700,000 km²)")

print("\nKey output locations:")
print(f"  - Basin shapefiles: {basin_path}")
print(f"  - River network: {network_path}")
print(f"  - HRU shapefiles: {hru_path}")
print(f"  - Model settings: {project_dir}/settings/{confluence.config['HYDROLOGICAL_MODEL']}/")
print(f"  - Future simulation results: {project_dir}/simulations/{confluence.config['EXPERIMENT_ID']}/")

print("\n🎉 You've successfully scaled from watershed to continent!")